In [1]:
# default_exp trainer

In [1]:
import gcp_runner.core
gcp_runner.core.export_and_reload_all(silent=True)

In [2]:
#export
from criteo_nbdev.constants import *
import datetime
import tensorflow as tf

class TrainTimeCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = datetime.datetime.now()

    def on_epoch_end(self, epoch, logs=None):
        logging.info('\nepoch train time: (hh:mm:ss.ms) {}'.format(
            datetime.datetime.now() - self.epoch_start_time))
        if not self.params is None:
            if 'steps' in self.params and self.params['steps']:
                epoch_milliseconds = (datetime.datetime.now(
                ) - self.epoch_start_time).total_seconds() * 1000
                logging.info(
                    '{} ms/step'.format(epoch_milliseconds / self.params['steps']))
                if BATCH_SIZE is not None:
                    logging.info('{} microseconds/example'.format(
                        1000 * epoch_milliseconds / self.params['steps'] / BATCH_SIZE))

In [3]:
#export
from criteo_nbdev.constants import *
import tensorflow as tf


def create_categorical_feature_column_with_hash_bucket(corpus_dict, key):
    corpus_size = len(corpus_dict[key])
    hash_bucket_size = min(corpus_size, 100000)
    categorical_feature_column = tf.feature_column.categorical_column_with_hash_bucket(
        key,
        hash_bucket_size,
        dtype=tf.dtypes.string
    )
    logging.info('categorical column %s hash_bucket_size %d',
                 key, hash_bucket_size)
    return categorical_feature_column


def create_categorical_feature_column_with_vocabulary_list(corpus_dict, key):
    corpus_size = len(corpus_dict[key])
    categorical_feature_column = tf.feature_column.categorical_column_with_vocabulary_list(
        key,
        list(corpus_dict[key].keys()),
        dtype=tf.dtypes.string,
        num_oov_buckets=corpus_size
    )
    logging.info(
        'categorical column with vocabular %s corpus_size %d', key, corpus_size)

    return categorical_feature_column

def create_embedding(vocabulary_size_dict, key, categorical_feature_column):
    vocabulary_size = vocabulary_size_dict[key]
    if vocabulary_size < 10:
        logging.info(
            'categorical column %s vocabulary_size %d - creating indicator column', key, vocabulary_size)
        return tf.feature_column.indicator_column(categorical_feature_column)

    embedding_dimension = int(min(50, math.floor(6 * vocabulary_size**0.25)))
    embedding_feature_column = tf.feature_column.embedding_column(
        categorical_feature_column,
        embedding_dimension)
    return embedding_feature_column

def create_linear_feature_columns():
    return list(tf.feature_column.numeric_column(field.name, dtype=tf.dtypes.float32) for field in CSV_SCHEMA if field.field_type == 'INTEGER' and field.name != 'label')

def create_categorical_embeddings_feature_columns(corpus_dict, vocabulary_size_dict, embeddings_mode: EMBEDDINGS_MODE_TYPE):
    if embeddings_mode == EMBEDDINGS_MODE_TYPE.none:
        return []
    elif embeddings_mode == EMBEDDINGS_MODE_TYPE.hashbucket:
        return list(create_embedding(
            vocabulary_size_dict,
            key,
            create_categorical_feature_column_with_hash_bucket(corpus_dict, key))
            for key, _ in corpus_dict.items())
    elif embeddings_mode == EMBEDDINGS_MODE_TYPE.vocabular:
        return list(create_embedding(
            vocabulary_size_dict,
            key,
            create_categorical_feature_column_with_vocabulary_list(corpus_dict, key))
            for key, _ in corpus_dict.items())
    else:
        raise ValueError('invalid embedding_mode: {}'.format(embedding_mode))


In [4]:
#export
import criteo_nbdev.data_reader

def create_feature_columns(embedding_mode: EMBEDDINGS_MODE_TYPE):
    corpus_dict = criteo_nbdev.data_reader.get_corpus_dict()
    vocabulary_size_dict = criteo_nbdev.data_reader.get_vocabulary_size_dict()
    feature_columns = []
    feature_columns.extend(create_linear_feature_columns())
    feature_columns.extend(
        create_categorical_embeddings_feature_columns(corpus_dict, vocabulary_size_dict, embedding_mode))
    return feature_columns

In [5]:
#export

from criteo_nbdev.constants import *
import tensorflow as tf

def create_keras_model_sequential():
    feature_columns = create_feature_columns(EMBEDDINGS_MODE_TYPE.hashbucket)

    feature_layer = tf.keras.layers.DenseFeatures(
        feature_columns, name="feature_layer")
    Dense = tf.keras.layers.Dense
    Dropout = tf.keras.layers.Dropout
    BatchNormalization = tf.keras.layers.BatchNormalization
    model = tf.keras.Sequential(
        [
            feature_layer,
            Dropout(0.3),
            Dense(598, activation=tf.nn.relu),
            Dense(598, activation=tf.nn.relu),
            Dense(598, activation=tf.nn.relu),
            Dense(1, activation=tf.nn.sigmoid)
        ])

    logging.info('compiling sequential keras model')
    # Compile Keras model
    model.compile(
        # cannot use Adagrad with mirroredstartegy https://github.com/tensorflow/tensorflow/issues/19551
        # optimizer=tf.optimizers.Adagrad(learning_rate=0.05),
        optimizer=tf.optimizers.SGD(learning_rate=0.05),
        # optimizer=tf.optimizers.Adam(learning_rate=0.0005),
        # optimizer=tf.optimizers.Adam(),
        #optimizer=tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.1),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=['accuracy'])
    return model

In [6]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [7]:
#export
from criteo_nbdev.constants import *
from gcp_runner.ai_platform_constants import *
import criteo_nbdev.data_reader
import nbdev.imports
import tensorflow as tf
import logging
import math
import os

def train_and_evaluate_keras_model(
    model, 
    model_dir, 
    epochs,
    dataset_source: DATASET_SOURCE_TYPE,
    dataset_size: DATASET_SIZE_TYPE,
    embeddings_mode: EMBEDDINGS_MODE_TYPE,
    distribution_strategy: DistributionStrategyType):
    
    log_dir = os.path.join(model_dir, "logs")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,
        embeddings_freq=1)

    checkpoints_dir = os.path.join(model_dir, "checkpoints")
    # crashing https://github.com/tensorflow/tensorflow/issues/27688
    if not os.path.exists(checkpoints_dir):
        os.makedirs(checkpoints_dir)

    callbacks = []
    train_time_callback = TrainTimeCallback()

    if DistributionStrategyType == DistributionStrategyType.TPU_STRATEGY:
        # epoch and accuracy constants are not supported when training on TPU.
        checkpoints_file_path = checkpoints_dir + "/epochs_tpu.hdf5"
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            checkpoints_file_path, verbose=1, mode='max')
        callbacks = [tensorboard_callback,
                     checkpoint_callback, train_time_callback]
    else:
        if embeddings_mode == EMBEDDINGS_MODE_TYPE.manual or distribution_strategy == DistributionStrategyType.MULTI_WORKER_MIRRORED_STRATEGY:
            # accuracy fails for adagrad
            # for some reason accuracy is not available for EMBEDDINGS_MODE_TYPE.manual
            # for some reason accuracy is not available for MultiWorkerMirroredStrategy
            checkpoints_file_path = checkpoints_dir + \
                "/epochs:{epoch:03d}.hdf5"
        else:
            checkpoints_file_path = checkpoints_dir + \
                "/epochs:{epoch:03d}-accuracy:{accuracy:.3f}.hdf5"
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            checkpoints_file_path, verbose=1, mode='max')
        callbacks = [tensorboard_callback, checkpoint_callback, train_time_callback]

    verbosity = 1 if nbdev.imports.in_ipython() else 2
    logging.info('training keras model')
    training_ds = criteo_nbdev.data_reader.get_dataset(dataset_source, dataset_size, DATASET_TYPE.training, embeddings_mode).repeat(epochs)
    eval_ds = criteo_nbdev.data_reader.get_dataset(dataset_source, dataset_size, DATASET_TYPE.validation, embeddings_mode).repeat(epochs)
    
    # steps_per_epoch and validation_steps are required for MultiWorkerMirroredStrategy
    model.fit(
        training_ds,
        epochs=epochs,
        verbose=verbosity,
        callbacks=callbacks,
        steps_per_epoch=criteo_nbdev.data_reader.get_steps_per_epoch(dataset_size, DATASET_TYPE.training),
        validation_data=eval_ds,
        validation_steps=criteo_nbdev.data_reader.get_steps_per_epoch(dataset_size, DATASET_TYPE.validation))

    logging.info("done training keras model, evaluating model")
    loss, accuracy = model.evaluate(
        eval_ds, 
        verbose=verbosity, 
        steps=criteo_nbdev.data_reader.get_steps_per_epoch(dataset_size, DATASET_TYPE.validation), 
        callbacks=[tensorboard_callback])
    logging.info("Eval - Loss: {}, Accuracy: {}".format(loss, accuracy))
    logging.info(model.summary())
    logging.info("done evaluating keras model")
    return {'accuracy': accuracy, 'loss': loss}


In [8]:
#export

import gcp_runner.core
gcp_runner.core.export_and_reload_all(silent=True)

# def train_keras_sequential(strategy, model_dir):
#     return train_and_evaluate_keras_model(create_keras_model_sequential(), model_dir)

# train_keras_sequential(None, './models/model1')

def train_and_evaluate_keras_model_small(distribution_strategy=None, job_dir=None, **kwargs):
    print('distribution_strategy:')
    print(distribution_strategy)
    print('job_dir:')
    print(job_dir)
    print('kwargs:')
    print(kwargs)
    train_and_evaluate_keras_model(create_keras_model_sequential(), job_dir, 2, DATASET_SOURCE_TYPE.bq, DATASET_SIZE_TYPE.tiny, EMBEDDINGS_MODE_TYPE.hashbucket, None)
    


In [9]:
from criteo_nbdev.core import skip; skip()

train_and_evaluate_keras_model_small(job_dir='models/model6')

In [ ]:
from criteo_nbdev.core import skip; skip()

import gcp_runner.local_runner
gcp_runner.local_runner.run_python(train_and_evaluate_keras_model_small, python_binary='python3', job_dir='./models/model7')

In [ ]:
from criteo_nbdev.core import skip; skip()

import gcp_runner.local_runner
gcp_runner.local_runner.run_docker(
    train_and_evaluate_keras_model_small, 
    'gcr.io/alekseyv-scalableai-dev/criteo-nbdev', 
    build_docker_file='./Dockerfile', 
    job_dir='./models/model6')

In [ ]:
from criteo_nbdev.core import skip; skip()
# won't work until gcp_runner is published on pypi, also need to fix package dependencies

import gcp_runner.ai_platform_runner
gcp_runner.ai_platform_runner.run_package(
     train_and_evaluate_keras_model_small,
     'gs://alekseyv-scalableai-dev-criteo-model-bucket/test-job-dir',
     region='us-west1')

In [ ]:
#from criteo_nbdev.core import skip; skip()

import gcp_runner.ai_platform_runner
gcp_runner.ai_platform_runner.run_docker_image(
     train_and_evaluate_keras_model_small,
     'gs://alekseyv-scalableai-dev-criteo-model-bucket/models/model_mirrored_strategy_{username}_{datetime}',
     master_image_uri='gcr.io/alekseyv-scalableai-dev/criteo-nbdev',
     build_docker_file='./Dockerfile',
     region='us-west1',
     distribution_strategy_type = gcp_runner.ai_platform_constants.DistributionStrategyType.MIRRORED_STRATEGY,
     use_distribution_strategy_scope=True)

To see job output in tensorboard, run following command:
tensorboard --logdir=%s gs://alekseyv-scalableai-dev-criteo-model-bucket/models/model_mirrored_strategy_alekseyv_20200401_191035
Building Docker image:
docker build -f ./Dockerfile -t gcr.io/alekseyv-scalableai-dev/criteo-nbdev ./
Sending build context to Docker daemon  2.976MB
Step 1/14 : FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-1
 ---> 4c5f4efaf2dd
Step 2/14 : WORKDIR /root
 ---> Using cache
 ---> 36acbff89f75
Step 3/14 : ENV PROJECT_ID=alekseyv-scalableai-dev
 ---> Using cache
 ---> 4287a724b19d
Step 4/14 : ENV GOOGLE_APPLICATION_CREDENTIALS=/root/service_account_key.json
 ---> Using cache
 ---> c5492faca793
Step 5/14 : COPY service_account_key.json /root/
 ---> Using cache
 ---> 45bb06aae86f
Step 6/14 : RUN pip install nbdev
 ---> Using cache
 ---> b2097138828d
Step 7/14 : ENV KMP_AFFINITY=""
 ---> Using cache
 ---> 3c6c699cb173
Step 8/14 : ENV TF_DISABLE_MKL=1
 ---> Using cache
 ---> bcc1fb62d4ef
Step 9/14 : ADD "

In [ ]:
#from criteo_nbdev.core import skip; skip()
import gcp_runner.ai_platform_constants

import gcp_runner.kubernetes_runner
gcp_runner.kubernetes_runner.run_docker_image(
     train_and_evaluate_keras_model_small,
     'gs://alekseyv-scalableai-dev-criteo-model-bucket/test-job-dir/model_mirrored_strategy_{username}_{datetime}',
     image_uri='gcr.io/alekseyv-scalableai-dev/criteo-nbdev',
     build_docker_file='./Dockerfile',
     distribution_strategy_type = gcp_runner.ai_platform_constants.DistributionStrategyType.MIRRORED_STRATEGY,
     use_distribution_strategy_scope=True)

In [ ]:
import gcp_runner.core
gcp_runner.core.export_and_reload_all(silent=True)